In [ ]:
for rng_itrs in [0]: # Put deired list of RNG values
    import numpy as np
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    torch.cuda.empty_cache()
    import matplotlib.pyplot as plt
    
    import snntorch as snn
    from snntorch import surrogate

    from timeit import default_timer
    from pytorch_wavelets import DWT, IDWT # (or import DWT, IDWT)

    torch.cuda.empty_cache()
    torch.manual_seed(rng_itrs)
    np.random.seed(rng_itrs)


    from utilities_0 import *
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#     from utilities_1 import *
#     device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

    # %%
    """ Def: 2d Wavelet layer """
    class WaveConv2d(nn.Module):
        def __init__(self, in_channels, out_channels, level, dummy):
            super(WaveConv2d, self).__init__()

            self.in_channels = in_channels
            self.out_channels = out_channels
            self.level = level
            self.dwt_ = DWT(J=self.level, mode='symmetric', wave='db6').to(dummy.device)
            self.mode_data, _ = self.dwt_(dummy)
            self.modes1 = self.mode_data.shape[-2]
            self.modes2 = self.mode_data.shape[-1]

            self.scale = (1 / (in_channels * out_channels))
            self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2))
            self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2))
            
        # Convolution
        def mul2d(self, input, weights):
            # (batch, in_channel, x,y ), (in_channel, out_channel, x,y) -> (batch, out_channel, x,y)
            return torch.einsum("bixy,ioxy->boxy", input, weights)

        def forward(self, x):
            batchsize = x.shape[0]
            # Compute single tree Discrete Wavelet coefficients using some wavelet
            dwt = DWT(J=self.level, mode='symmetric', wave='db6').to(x.device)
            x_ft, x_coeff = dwt(x)

            # Multiply relevant Wavelet modes
            out_ft = torch.zeros(batchsize, self.out_channels,  x_ft.shape[-2], x_ft.shape[-1], device=x.device)
            out_ft = self.mul2d(x_ft, self.weights1)
            # Multiply the finer wavelet coefficients
            x_coeff[-1][:,:,0,:,:] = self.mul2d(x_coeff[-1][:,:,0,:,:].clone(), self.weights2)
            
            # Return to physical space        
            idwt = IDWT(mode='symmetric', wave='db6').to(x.device)
            x = idwt((out_ft, x_coeff))
            return x

    """ The forward operation """
    class WNO2d(nn.Module):
        def __init__(self, width, level, dummy_data):
            super(WNO2d, self).__init__()

            self.level = level
            self.dummy_data = dummy_data
            self.width = width
            self.padding = 1 # pad the domain when required
            self.fc0 = nn.Linear(3, self.width) # input channel is 3: (a(x, y), x, y)

            self.conv0 = WaveConv2d(self.width, self.width, self.level, self.dummy_data)
            self.conv1 = WaveConv2d(self.width, self.width, self.level, self.dummy_data)
            self.conv2 = WaveConv2d(self.width, self.width, self.level, self.dummy_data)
            self.conv3 = WaveConv2d(self.width, self.width, self.level, self.dummy_data)
            self.w0 = nn.Conv2d(self.width, self.width, 1)
            self.w1 = nn.Conv2d(self.width, self.width, 1)
            self.w2 = nn.Conv2d(self.width, self.width, 1)
            self.w3 = nn.Conv2d(self.width, self.width, 1)

            self.fc1 = nn.Linear(self.width, 128)
            self.fc2 = nn.Linear(128, 1)
            
            beta = torch.rand(52,52)
            thr = torch.rand(52,52)   
            self.lif1 = snn.Leaky(beta=beta, threshold=thr, reset_mechanism='zero',
                                  learn_beta=True, learn_threshold=True,
                                  spike_grad=surrogate.fast_sigmoid())
            
            beta = torch.rand(52,52)
            thr = torch.rand(52,52)   
            self.lif2 = snn.Leaky(beta=beta, threshold=thr, reset_mechanism='zero',
                                  learn_beta=True, learn_threshold=True,
                                  spike_grad=surrogate.fast_sigmoid())
            
            beta = torch.rand(52,52)
            thr = torch.rand(52,52)    
            self.lif3 = snn.Leaky(beta=beta, threshold=thr, reset_mechanism='zero',
                                  learn_beta=True, learn_threshold=True,
                                  spike_grad=surrogate.fast_sigmoid())        
            
            beta = torch.rand(128)
            thr = torch.rand(128)   
            self.lif4 = snn.Leaky(beta=beta, threshold=thr, reset_mechanism='zero',
                                  learn_beta=True, learn_threshold=True,
                                  spike_grad=surrogate.fast_sigmoid())

        def forward(self, x):
            n_spikes = 1
            x_spiketime = torch.empty([x.shape[0], 51, 51, 128, n_spikes]).to(x.device)

            mem1 = self.lif1.init_leaky()
            mem2 = self.lif2.init_leaky()
            mem3 = self.lif3.init_leaky()
            mem4 = self.lif4.init_leaky()

            grid = self.get_grid(x.shape, x.device)
            x = torch.cat((x, grid), dim=-1)
            inputs = x

            s1 = 0
            s2 = 0
            s3 = 0
            s4 = 0        
            for i in range(0,n_spikes):

                x = self.fc0(inputs)
                x = x.permute(0, 3, 1, 2)
                x = F.pad(x, [0,self.padding, 0,self.padding]) # padding, if required

                x1 = self.conv0(x)
                x2 = self.w0(x)
                x = x1 + x2
                spike,mem1 = self.lif1(x, mem1)
                x = (x*spike)

                s1 += spike.sum()/(x.shape[0]*64*52*52)

                x1 = self.conv1(x)
                x2 = self.w1(x)
                x = x1 + x2
                spike,mem2 = self.lif2(x, mem2)
                x = (x*spike)

                s2 += spike.sum()/(x.shape[0]*64*52*52)

                x1 = self.conv2(x)
                x2 = self.w2(x)
                x = x1 + x2
                spike,mem3 = self.lif3(x, mem3)
                x = (x*spike)

                s3 += spike.sum()/(x.shape[0]*64*52*52)

                x1 = self.conv3(x)
                x2 = self.w3(x)
                x = x1 + x2

                x = x[..., :-self.padding, :-self.padding] # removing padding, when applicable
                x = x.permute(0, 2, 3, 1)

                x = self.fc1(x)
                spike,mem4 = self.lif4(x, mem4)
                x = (x*spike)

                s4 += spike.sum()/(x.shape[0]*51*51*128)

                x_spiketime[:,:,:,:,i] = x

            x = torch.mean(x_spiketime, 4)
            x = self.fc2(x)

            return s1/n_spikes, s2/n_spikes, s3/n_spikes, s4/n_spikes, x

        def get_grid(self, shape, device):
            # The grid of the solution
            batchsize, size_x, size_y = shape[0], shape[1], shape[2]
            gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
            gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
            gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
            gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
            return torch.cat((gridx, gridy), dim=-1).to(device)

    # %%

    """ Model configurations """

    PATH = '/DATA/SG/WNO/data/Darcy_Triangular_FNO.mat'
    ntrain = 1900
    ntest = 100

    batch_size = 25
    learning_rate = 0.001

    epochs = 500
    step_size = 50
    gamma = 0.75

    level = 3
    width = 64

    r = 2
    h = int(((101 - 1)/r) + 1)
    s = h

    # %%
    """ Read data """
    reader = MatReader(PATH)
    x_train = reader.read_field('boundCoeff')[:ntrain,::r,::r][:,:s,:s]
    y_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s]

    x_test = reader.read_field('boundCoeff')[-ntest:,::r,::r][:,:s,:s]
    y_test = reader.read_field('sol')[-ntest:,::r,::r][:,:s,:s]

    x_normalizer = UnitGaussianNormalizer(x_train)
    x_train = x_normalizer.encode(x_train)
    x_test = x_normalizer.encode(x_test)

    y_normalizer = UnitGaussianNormalizer(y_train)
    y_train = y_normalizer.encode(y_train)

    x_train = x_train.reshape(ntrain,s,s,1)
    x_test = x_test.reshape(ntest,s,s,1)

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    # %%
    """ The model definition """
    model = WNO2d(width, level, x_train.permute(0,3,1,2)).to(device)
    print(count_params(model))

    from torchinfo import summary
    print(summary(model, input_size=(batch_size, s, s, 1)))

    # %%

    model = WNO2d(width, level, x_train.permute(0,3,1,2)).to(device)

    """ Training and testing """
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    train_loss = torch.zeros(epochs)
    test_loss = torch.zeros(epochs)
    myloss = LpLoss(size_average=False)
    y_normalizer.cuda()
    for ep in range(epochs):
        model.train()
        t1 = default_timer()
        train_l2 = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            s1,s2,s3,s4,out = model(x)
            out = out.reshape(batch_size, s, s)
            out = y_normalizer.decode(out)
            y = y_normalizer.decode(y)

            loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
            
            nl = loss + (s1+s2+s3+s4) # weights for different losses can be changed here
            nl.backward()
            optimizer.step()
            train_l2 += loss.item()

        scheduler.step()
        model.eval()
        test_l2 = 0.0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)

                s1,s2,s3,s4,out = model(x)
                out = out.reshape(batch_size, s, s)
                out = y_normalizer.decode(out)

                test_l2 += myloss(out.view(batch_size,-1), y.view(batch_size,-1)).item()

        train_l2/= ntrain
        test_l2 /= ntest

        train_loss[ep] = train_l2
        test_loss[ep] = test_l2

        t2 = default_timer()
        print('%5d %15.4f %15.4e %15.4e %7.4f %7.4f %7.4f %7.4f'%(ep, t2-t1, train_l2, test_l2, s1, s2, s3, s4))

#     # %%

    filename = './model/RNG_'+str(rng_itrs)+'_Model_2DDarcyTriangular_1900TDS_WNO_VSNN_New_LOSS_1_1_LearnBeTh.pt'
    torch.save(model.state_dict(), filename)

    # %%

    s = h

    """ Prediction """

    pred = torch.zeros([y_test.shape[0], y_test.shape[1], y_test.shape[2]])
    myloss = LpLoss(size_average=False)

    filename = './model/RNG_'+str(rng_itrs)+'_Model_2DDarcyTriangular_1900TDS_WNO_VSNN_New_LOSS_1_1_LearnBeTh.pt'

    loaded_model = WNO2d(width, level, x_train.permute(0,3,1,2)).to(device)
    loaded_model.load_state_dict(torch.load(filename))

    index = 0
    test_e = torch.zeros(y_test.shape[0])
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
    
    ts1 = 0
    ts2 = 0
    ts3 = 0
    ts4 = 0
    with torch.no_grad():
        for x, y in test_loader:
            test_l2 = 0
            x, y = x.to(device), y.to(device)

            s1,s2,s3,s4,out = loaded_model(x)
            out = out.reshape(1, s, s)
            out = y_normalizer.decode(out)
            pred[index,:,:] = out

            test_l2 = myloss(out.reshape(1, s, s), y.reshape(1, s, s)).item()
            test_e[index] = test_l2

            # print(index, test_l2)
            index = index + 1
            ts1 += s1
            ts2 += s2
            ts3 += s3
            ts4 += s4

    print('Mean Testing Error:', 100*torch.mean(test_e).numpy(), '%')
    print('%.4f %.4f %.4f %.4f'%(100*ts1.item()/ntest, 100*ts2.item()/ntest, 100*ts3.item()/ntest, 100*ts4.item()/ntest))
    
    # %%

    m = pred.numpy()

    plt.figure(constrained_layout=False, figsize = (7, 14))
    plt.subplots_adjust(hspace=0.75)
    index = 0

    for value in range(y_test.shape[0]):
        if value % 60 == 1:
            plt.subplot(5,2, index+1)
            plt.imshow(y_test[value,:,:], origin='lower', cmap='seismic',
                       vmin = np.min(y_test[value,:,:].numpy()), vmax = np.max(y_test[value,:,:].numpy()))
            plt.colorbar()
            plt.title('Actual')

            plt.subplot(5,2, index+1+2)
            plt.imshow(m[value,:,:], origin='lower', cmap='seismic',
                       vmin = np.min(y_test[value,:,:].numpy()), vmax = np.max(y_test[value,:,:].numpy()))
            plt.colorbar()
            plt.title('Mean')

            index = index + 1
    
    plt.show()
    print((100*myloss(y_test, torch.tensor(m)).item())/40)
    print(100*torch.sum((y_test-torch.tensor(m))**2).item())

    # %%

    for marker in ([1-1,5-1,15-1,29-1]):
        plt.figure(constrained_layout=False, figsize = (10, 3))
        plt.subplots_adjust(wspace=0.3)
        index = 0
        for value in range(y_test.shape[0]):
            if value % 60 == 1:
                plt.subplot(1,2, index+1)
                plt.plot(y_test[value,marker,:],'r')
                plt.plot(m[value,marker,:],'b')
                index = index + 1
        plt.show()

In [ ]:
# PLOTS

In [ ]:
    """ Plotting """

    plt.figure(figsize = (14, 2))
    plt.subplots_adjust(hspace=0.5)
    index = 0
    for value in  [0,10]:
        plt.subplot(1,4, index*2+1)
        plt.imshow(y_test[value,:,:], label='True', cmap='seismic', origin='lower')
        # plt.title('Actual')
        plt.xticks([0,25,50],[0,0.5,1], fontsize = 14)
        plt.yticks([0,25,50],[0,0.5,1], fontsize = 14)
        cbar = plt.colorbar()
        cbar.ax.tick_params(labelsize=14)

        xcord=np.array([26,26,24,24]); ycord=np.array([0,21,21,0])
        plt.axis('square'); plt.fill_between(xcord,ycord, color = 'white')
        
        xcord=np.array([0,0,25,0]); ycord=np.array([0,50,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        xcord=np.array([50,50,25,50]); ycord=np.array([0,50,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        
        xcord=np.array([0,0,50,50,0]); ycord=np.array([50,44,44,50,50])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        

        index += 1
        
    plt.savefig('/DATA/SG/NINN_WNO/Final_codes/Plots/DTri_GT.pdf')
    plt.savefig('/DATA/SG/NINN_WNO/Final_codes/Plots/DTri_GT.png')    
    plt.show()
    
    """ Plotting """

    plt.figure(figsize = (14, 2))
    # plt.subplots_adjust(hspace=0.7)
    index = 0
    for value in [0,10]:
        plt.subplot(1,4, index*2+1)
        plt.imshow(m[value,:,:], cmap='seismic', origin='lower')
        # plt.title('Identified')
        plt.xticks([0,25,50],[0,0.5,1], fontsize = 14)
        plt.yticks([0,25,50],[0,0.5,1], fontsize = 14)
        cbar = plt.colorbar()
        cbar.ax.tick_params(labelsize=14)

        xcord=np.array([26,26,24,24]); ycord=np.array([0,21,21,0])
        plt.axis('square'); plt.fill_between(xcord,ycord, color = 'white')
        
        xcord=np.array([0,0,25,0]); ycord=np.array([0,51,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        xcord=np.array([51,51,25,51]); ycord=np.array([0,51,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        
        xcord=np.array([0,0,51,51,0]); ycord=np.array([51,44,44,51,51])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
                

        plt.subplot(1,4, index*2+2)
        plt.imshow(np.abs(y_test[value,:,:]-m[value,:,:]), cmap='seismic', origin='lower')
        # plt.title('Error')
        plt.xticks([0,25,50],[0,0.5,1], fontsize = 14)
        plt.yticks([0,25,50],[0,0.5,1], fontsize = 14)
        cbar = plt.colorbar()
        cbar.formatter.set_powerlimits((0, 0))
        cbar.update_ticks()
        cbar.ax.tick_params(labelsize=14)
        cbar.ax.yaxis.get_offset_text().set(size=14)
        plt.margins(0)
        
        xcord=np.array([26,26,24,24]); ycord=np.array([0,21,21,0])
        plt.axis('square'); plt.fill_between(xcord,ycord, color = 'white')
        
        xcord=np.array([0,0,25,0]); ycord=np.array([0,50,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        xcord=np.array([50,50,25,50]); ycord=np.array([0,50,44,0])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        
        xcord=np.array([0,0,50,50,0]); ycord=np.array([50,44,44,50,50])
        plt.axis('square'); plt.fill_between(xcord,ycord,color='white')
        
        index = index + 1
    
    plt.savefig('/DATA/SG/NINN_WNO/Final_codes/Plots/DTri_VSNN_LearnBeTh_NL_1_1.pdf')
    plt.savefig('/DATA/SG/NINN_WNO/Final_codes/Plots/DTri_VSNN_LearnBeTh_NL_1_1.png')    
    plt.show()